In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from scipy.sparse import hstack, csr_matrix

# Load data
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

# Initialize vectorizers and label encoders
n_features = 2**10
vectorizers = {
    'prompt': HashingVectorizer(n_features=n_features),
    'response_a': HashingVectorizer(n_features=n_features),
    'response_b': HashingVectorizer(n_features=n_features)
}
model_encoder = LabelEncoder()

# Encode model identifiers
train['model_a_encoded'] = model_encoder.fit_transform(train['model_a'])
train['model_b_encoded'] = model_encoder.transform(train['model_b'])

# Process text data into vectors
def process_and_concat_features(data, vectorizers):
    features_list = []
    for column, vectorizer in vectorizers.items():
        print(f"Vectorizing '{column}'...")
        transformed_data = vectorizer.transform(data[column])
        features_list.append(transformed_data)
    final_features = hstack(features_list)  # Keep as sparse matrix
    return final_features

train_features = process_and_concat_features(train, vectorizers)

# Combine model identifiers with text features
model_features = csr_matrix(train[['model_a_encoded', 'model_b_encoded']])
X_combined = hstack([model_features, train_features])

# Encode target variable
train['winner'] = train.apply(lambda row: 'model_a' if row['winner_model_a'] == 1 else 'model_b' if row['winner_model_b'] == 1 else 'tie', axis=1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train['winner'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_encoded, test_size=0.2, random_state=42)

# Convert the data to DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set up parameters
params = {
    'objective': 'multi:softprob',
    'num_class': len(np.unique(y_encoded)),
    'eval_metric': 'mlogloss',
    'tree_method':'hist',
    'device':'cuda'
}

# Train the model
num_boost_round = 100
bst = xgb.train(params, dtrain, num_boost_round)

# Predict the probabilities
pred_probs = bst.predict(dtest)
predictions = np.argmax(pred_probs, axis=1)

# Evaluate the model
from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(y_test, predictions)
print(f"Test Accuracy: {test_accuracy:.2f}")


In [ ]:
# Load data
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
test_features = process_and_concat_features(test, vectorizers)

dtest = xgb.DMatrix(test_features)

# Predict the probabilities
pred_probs = bst.predict(dtest)

# Create DataFrame with the prediction probabilities for each class
df_submission = pd.DataFrame(pred_probs, columns=label_encoder.classes_)
df_submission.insert(0, 'id', test['id'])
df_submission.columns = ['id', 'winner_model_a', 'winner_model_b', 'winner_tie']

# Save predictions to CSV
df_submission.to_csv('submission.csv', index=False)
print(df_submission.head())
